In [1]:
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using DifferentialEquations
using MeshIO

## Importing ShadowHand URDF

### URDF Parsing

In [11]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Hand Visualization

In [12]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)

# Display figure
display(fig)

GLMakie.Screen(...)

### Kinematic Computations

In [4]:
function struct_fields_and_types(T::Type)
    for (name, type) in zip(fieldnames(T), fieldtypes(T))
        println(name, "::", type)
    end
end

kcache = new_kinematics_cache(m)  
rh_ff_tip_frame_id = get_compiled_frameID(m, "rh_fftip")
ff_tip_transform = get_transform(kcache, rh_ff_tip_frame_id)
display(ff_tip_transform)

rh_mf_tip_frame_id = get_compiled_frameID(m, "rh_mftip")
mf_tip_transform = get_transform(kcache, rh_mf_tip_frame_id)
display(mf_tip_transform)

struct_fields_and_types(typeof(ff_tip_transform))

display(ff_tip_transform.origin)
display(mf_tip_transform.origin)

fftip_to_mftip_transform = inv(ff_tip_transform) * mf_tip_transform

Transform{Float64}([0.033, -0.01, 0.43801000000000007], Rotor{Float64}(1.0, [0.0, 0.0, 0.0]))

Transform{Float64}([0.011, -0.01, 0.44201000000000007], Rotor{Float64}(1.0, [0.0, 0.0, 0.0]))

origin::SVector{3, Float64}
rotor::Rotor{Float64}


3-element SVector{3, Float64} with indices SOneTo(3):
  0.033
 -0.01
  0.43801000000000007

3-element SVector{3, Float64} with indices SOneTo(3):
  0.011
 -0.01
  0.44201000000000007

Transform{Float64}([-0.022000000000000002, 0.0, 0.0040000000000000036], Rotor{Float64}(1.0, [0.0, -0.0, 0.0]))

## Creating the Virtual Mechanism System

### Gravity Compensation and Joint Damping

In [5]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Virtual Mechanism Initialization

In [6]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

add_coordinate!(vms, FrameOrigin(".robot.rh_palm");        id="Target position")
add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "Target position"); id="Position error")
add_coordinate!(vms, CoordDifference(".robot.rh_mftip_mass_coord",".robot.rh_fftip_mass_coord"); id="ff_mf_diff")
add_coordinate!(vms, ConstCoord(fftip_to_mftip_transform.origin); id ="ff_mf_vector")
add_coordinate!(vms, CoordDifference("ff_mf_diff", "ff_mf_vector"); id = "ff_mf")

"ff_mf"

In [7]:
K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
add_component!(vms, LinearSpring(K, "Position error");           id="Linear Spring")
D = SMatrix{3, 3}(100., 0., 0., 0., 100.0, 0., 0., 0., 100.)
add_component!(vms, LinearDamper(D, "Position error");           id="Linear Damper")

"Linear Damper"

In [8]:
add_component!(vms, LinearSpring(K, "ff_mf"); id="ff_mf_spring")
add_component!(vms, LinearDamper(D, "ff_mf"); id="ff_mf_damper")

"ff_mf_damper"

## Simulating the Robot

### Setting Up the Simulation

In [9]:
disturbance_func(t) = mod(t, 6) < 3 ? SVector(0., 0., 0.) : SVector(0., 0., 10.)

f_setup(cache) = get_compiled_coordID(cache, ".robot.rh_fftip_mass_coord")

function f_control(cache, t, args, extra)
    tcp_pos_coord_id = args
    F = disturbance_func(t)
    uᵣ, uᵥ = get_u(cache)
    z = configuration(cache, tcp_pos_coord_id)
    J = jacobian(cache, tcp_pos_coord_id)
    mul!(uᵣ, J', F)
    nothing
end

tspan = (0., 12.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with impedance control."

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with impedance control.
ODE   0%|█                                              |  ETA: N/A
ODE   0%|█                                              |  ETA: 0:07:10
ODE   1%|█                                              |  ETA: 0:04:58
ODE   1%|█                                              |  ETA: 0:04:15
ODE   2%|█                                              |  ETA: 0:03:55
ODE   2%|██                                             |  ETA: 0:03:42
ODE   3%|██                                             |  ETA: 0:03:33
ODE   3%|██                                             |  ETA: 0:03:25
ODE   4%|██                                             |  ETA: 0:03:20
ODE   4%|███                                            |  ETA: 0:03:15
ODE   5%|███                                            |  ETA: 0:03:12
ODE   5%|███                                            |  ETA: 0:03:09
ODE   6%|███                                            |  ETA: 0:03:06
ODE   6%|███

### Visualizing the Results

In [10]:
fig = Figure(size = (720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls, camera=:perspective, center=false)
cam.lookat[] = [0.25, 0.5, 0.1]
cam.eyeposition[] = [-0.4, -0.8, 0.5]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(robot)))
robotvisualize!(ls, plotting_kcache;)

tcp_pos_id = get_compiled_coordID(plotting_kcache[], "rh_fftip_mass_coord")
tcp_pos = map(plotting_kcache) do kcache
    Point3f(configuration(kcache, tcp_pos_id))
end
force = map(t -> 0.01 * Vec3f(disturbance_func(t)), plotting_t)
arrowsize = map(f -> 0.1*(f'*f)^(0.25), force)
arrows!(ls, map(p -> [p], tcp_pos), map(f -> [f], force); color = :red, arrowsize)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_two_fingers_grasp.mp4")
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);